In [126]:
import random
from datetime import timedelta

import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 1000)


In [154]:

def create_dummy_data() -> pd.DataFrame:
    date_index = pd.date_range(start="2023-01-01", end="2023-01-05", freq="s")

    columns = ["data1", "data2", "data3"]
    data = {}
    for col in columns:
        data.update({col: np.random.randn(len(date_index))})
    df = pd.DataFrame(index=date_index, data=data)
    df["time_delta"] = [timedelta(seconds=random.randrange(1, 10)) for _ in range(len(df))]
    df["time_delta_cumsum"] = df["time_delta"].cumsum()
    df["Date_Time"] = df.index + df["time_delta_cumsum"]
    df.drop(["time_delta", "time_delta_cumsum"], axis=1, inplace=True)
    # df["time_delta"] = df["time_delta"].dt.seconds
    df.set_index("Date_Time", inplace=True)

    return df

def resample_dataframe(df: pd.DataFrame, interval: str = "30s") -> pd.DataFrame:
    re_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq=interval)
    re_df = pd.DataFrame(index=re_index)
    result = pd.merge_asof(
        re_df,
        df,
        left_index=True,
        right_index=True,
        direction="backward",
        tolerance=pd.Timedelta(seconds=15),
    )
    return result


In [155]:
df = create_dummy_data()
print(df.head(50))
print(resample_dataframe(df).head(50))

,data1,data2,data3
2023-01-01 00:00:07,-0.511768,-2.069940,0.031558
2023-01-01 00:00:37,-1.671503,-1.450244,-1.228165
2023-01-01 00:01:07,0.205969,-0.644198,0.138027
2023-01-01 00:01:37,-1.529062,1.287975,0.513107
2023-01-01 00:02:07,-0.064317,2.883432,-0.236760
...,...,...,...
2023-01-24 23:37:37,0.408291,-0.818317,0.586068
2023-01-24 23:38:07,-0.752498,0.680554,-2.356793
2023-01-24 23:38:37,-0.220667,-1.185959,-1.475818
2023-01-24 23:39:07,1.029231,-0.115837,-1.938857


In [156]:

df

,data1,data2,data3
Date_Time,,,
2023-01-01 00:00:07,-0.511768,-2.069940,0.031558
2023-01-01 00:00:09,1.830868,-1.034832,-0.672036
2023-01-01 00:00:15,0.976409,-0.644013,1.753451
2023-01-01 00:00:18,0.307812,-0.092997,-2.796929
2023-01-01 00:00:28,-1.671503,-1.450244,-1.228165
...,...,...,...
2023-01-24 23:39:21,0.801640,-0.100931,0.428326
2023-01-24 23:39:23,0.737547,-0.071194,0.603648
2023-01-24 23:39:31,-1.858396,-0.105751,-0.549258
